In [1]:
# Cell 1 - Import thư viện cần thiết
import pandas as pd
import numpy as np
import json
import re
import os
import glob
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
warnings.filterwarnings('ignore')

In [2]:
# Cell 2 - Vietnamese NLP Functions
class VietnameseNLP:
    def __init__(self):
        self.stopwords = {
            'và', 'của', 'có', 'là', 'trong', 'với', 'được', 'cho', 'từ', 'các', 'một', 'những',
            'này', 'đó', 'khi', 'để', 'không', 'về', 'sau', 'trước', 'hay', 'hoặc', 'nếu', 'như'
        }
        
    def normalize_vietnamese(self, text):
        return text.lower() if isinstance(text, str) else ""
    
    def remove_stopwords(self, text):
        if not isinstance(text, str):
            return ""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def clean_text_advanced(self, text, remove_stopwords=True, normalize=True):
        if not isinstance(text, str):
            return ""
        text = re.sub(r'[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        if normalize:
            text = self.normalize_vietnamese(text)
        if remove_stopwords:
            text = self.remove_stopwords(text)
        return text

def clean_vietnamese_text(text, remove_stopwords=True, normalize=True):
    nlp = VietnameseNLP()
    return nlp.clean_text_advanced(text, remove_stopwords, normalize)

In [3]:
# Cell 3 - Utility Functions
def convert_latex_to_text(text):
    if not isinstance(text, str):
        return text
    text = text.replace('\\n', '\n')
    replacements = {
        r'\\frac\{([^}]+)\}\{([^}]+)\}': r'(\1)/(\2)',
        r'\^{([^}]+)}': r'^(\1)',
        r'_{([^}]+)}': r'_(\1)',
        r'\\times': '×', r'\\div': '÷', r'\\pm': '±',
    }
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text.strip()

def parse_question(question_full):
    lines = question_full.split('\n')
    question = lines[0]
    if question.startswith('Câu'):
        question = re.sub(r'^Câu \d+:\s*', '', question)
    
    options = []
    for line in lines[1:]:
        line = line.strip()
        if line and line.startswith(('A.', 'B.', 'C.', 'D.')):
            options.append(line)
    
    return question.strip(), options

class ScoreTracker:
    def __init__(self):
        self.correct = 0
        self.total = 0
    
    def add_result(self, is_correct):
        self.total += 1
        if is_correct:
            self.correct += 1
    
    def get_accuracy(self):
        return (self.correct / self.total * 100) if self.total > 0 else 0

In [4]:
# Cell 4 - Load dữ liệu VNHSGE
def load_vnhsge_data(data_folder='Dataset'):
    subjects = ['Biology', 'Chemistry', 'Physics']
    all_data = []
    
    for subject in subjects:
        subject_path = os.path.join(data_folder, subject)
        if not os.path.exists(subject_path):
            continue
            
        json_files = glob.glob(os.path.join(subject_path, "*.json"))
        
        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                for item in data:
                    if 'Question' in item and 'Choice' in item:
                        question_text, options = parse_question(item['Question'])
                        
                        question_data = {
                            'id': item.get('ID', ''),
                            'question': question_text,
                            'options': options,
                            'answer': item['Choice'],
                            'subject': subject.lower(),
                            'explanation': convert_latex_to_text(item.get('Explanation', ''))
                        }
                        all_data.append(question_data)
            except:
                continue
    
    return pd.DataFrame(all_data)

raw_data = load_vnhsge_data()
print(f"📊 Đã tải {len(raw_data)} câu hỏi")
print(f"📋 Cột dữ liệu: {raw_data.columns.tolist()}")

📊 Đã tải 600 câu hỏi
📋 Cột dữ liệu: ['id', 'question', 'options', 'answer', 'subject', 'explanation']


In [5]:
# Cell 5 - Phân tích dữ liệu cơ bản
print("📈 THỐNG KÊ DỮ LIỆU:")
print(f"Tổng số câu hỏi: {len(raw_data)}")
print(f"Số môn học: {raw_data['subject'].nunique()}")
print("\n📊 Phân phối theo môn:")
print(raw_data['subject'].value_counts())

print("\n🔍 Mẫu dữ liệu:")
raw_data.head()

📈 THỐNG KÊ DỮ LIỆU:
Tổng số câu hỏi: 600
Số môn học: 3

📊 Phân phối theo môn:
subject
biology      200
chemistry    200
physics      200
Name: count, dtype: int64

🔍 Mẫu dữ liệu:


,id,question,options,answer,subject,explanation
0,MET_Bio_IE_2019_1,Có thể sử dụng hóa chất nào sau đây để phát hi...,"[A. Dung dịch NaCl., B. Dung dịch Ca(OH)2., C....",B,biology,Có thể được sử dụng Ca(OH)2 để phát hiện quá t...
1,MET_Bio_IE_2019_2,Động vật nào sau đây trao đổi khí với môi trườ...,"[A. Châu chấu., B. Sư tử., C. Chuột., D. Ếch đ...",A,biology,Châu chấu trao đổi khí với môi trường thông qu...
2,MET_Bio_IE_2019_3,Axit amin là đơn phân cấu tạo nên phân tử nào ...,"[A. ADN., B. mARN., C. tARN., D. Prôtêin.]",D,biology,Axit amin là đơn phân cấu tạo nên phân tử Prôt...
3,MET_Bio_IE_2019_4,Phân tử nào sau đây trực tiếp làm khuôn cho qu...,"[A. ADN., B. mARN., C. tARN., D. rARN.]",B,biology,Phân tử mARM trực tiếp làm khuôn cho quá trình...
4,MET_Bio_IE_2019_5,Một phân tử ADN ở vi khuẩn có 10% số nuclêôtit...,"[A. 10%., B. 30%., C. 20%., D. 40%.]",D,biology,"Theo nguyên tắc bổ sung A = T, G = X nên %A +%..."


In [6]:
# Cell 6 - Kiểm tra chất lượng dữ liệu
print("🔍 KIỂM TRA CHẤT LƯỢNG DỮ LIỆU:")
print(f"Null values:")
print(raw_data.isnull().sum())

print(f"\n📏 Độ dài câu hỏi:")
question_lengths = raw_data['question'].str.len()
print(f"Trung bình: {question_lengths.mean():.1f} ký tự")
print(f"Min: {question_lengths.min()}, Max: {question_lengths.max()}")

print(f"\n📝 Phân phối đáp án:")
print(raw_data['answer'].value_counts())

🔍 KIỂM TRA CHẤT LƯỢNG DỮ LIỆU:
Null values:
id             0
question       0
options        0
answer         0
subject        0
explanation    0
dtype: int64

📏 Độ dài câu hỏi:
Trung bình: 167.3 ký tự
Min: 21, Max: 754

📝 Phân phối đáp án:
answer
A    188
C    144
B    140
D    128
Name: count, dtype: int64


In [7]:
# Cell 7 - DifficultyClassifier Class
class DifficultyClassifier:
    def __init__(self):
        self.text_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1, 2), min_df=2, max_df=0.9)
        self.model = RandomForestClassifier(n_estimators=50, max_depth=8, random_state=42, n_jobs=-1)
        self.is_trained = False
        self.evaluation_results = {}
    
    def _extract_features(self, question_text, options_text):
        full_text = question_text + " " + options_text
        full_lower = full_text.lower()
        
        patterns = {
            'analysis': ['phân tích', 'so sánh', 'đánh giá', 'giải thích'],
            'calculation': ['tính', 'toán', 'công thức', 'mol', 'gam'],
            'synthesis': ['tổng hợp', 'phản ứng', 'cơ chế', 'quá trình'],
            'evaluation': ['ảnh hưởng', 'tác động', 'nguyên nhân'],
            'definition': ['là gì', 'tên gọi', 'thuộc'],
            'identification': ['màu', 'trạng thái', 'tính chất']
        }
        
        features = []
        for category in ['analysis', 'calculation', 'synthesis', 'evaluation', 'definition', 'identification']:
            count = sum(1 for word in patterns[category] if word in full_lower)
            features.append(count)
        
        features.extend([
            len(question_text.split()),
            len(options_text.split()),
            full_text.count('.'),
            sum(1 for c in full_text if c in '+-*/=()$^_'),
            sum(1 for c in full_text if c.isupper()),
            sum(1 for w in full_text.split() if len(w) > 8)
        ])
        
        features.extend([
            1 if 'tại sao' in full_lower or 'vì sao' in full_lower else 0,
            1 if 'như thế nào' in full_lower else 0,
            1 if 'bao nhiêu' in full_lower else 0
        ])
        
        return np.array(features).reshape(1, -1)
    
    def _create_labels(self, data):
        difficulties = []
        for _, row in data.iterrows():
            options_text = ' '.join(row['options']) if row['options'] else ''
            features = self._extract_features(row['question'], options_text).flatten()
            
            score = (features[0] + features[1] + features[2]) * 2
            score -= (features[4] + features[5])
            score += features[6] * 0.1 + features[9] * 0.2
            score += features[12] + features[13] * 2
            
            if score <= 2:
                difficulty = 'easy'
            elif score <= 5:
                difficulty = 'medium'
            else:
                difficulty = 'hard'
            
            difficulties.append(difficulty)
        
        return difficulties

In [8]:
# Cell 8 - Khởi tạo và train DifficultyClassifier
difficulty_classifier = DifficultyClassifier()
print("🤖 Đang tạo nhãn độ khó...")
difficulties = difficulty_classifier._create_labels(raw_data)

print("📊 PHÂN PHỐI ĐỘ KHÓ:")
difficulty_counts = pd.Series(difficulties).value_counts()
print(difficulty_counts)
print(f"\nTỷ lệ phần trăm:")
print((difficulty_counts / len(difficulties) * 100).round(1))

🤖 Đang tạo nhãn độ khó...
📊 PHÂN PHỐI ĐỘ KHÓ:
hard      282
medium    202
easy      116
Name: count, dtype: int64

Tỷ lệ phần trăm:
hard      47.0
medium    33.7
easy      19.3
Name: count, dtype: float64


In [9]:
# Cell 9 - Phân tích features cho độ khó
print("🔍 PHÂN TÍCH FEATURES CHO ĐỘ KHÓ:")
sample_questions = raw_data.head(10)
for i, (_, row) in enumerate(sample_questions.iterrows()):
    options_text = ' '.join(row['options']) if row['options'] else ''
    features = difficulty_classifier._extract_features(row['question'], options_text).flatten()
    print(f"Câu {i+1}: {difficulties[i]} - Features: {features[:6]}")  # Show first 6 features


🔍 PHÂN TÍCH FEATURES CHO ĐỘ KHÓ:
Câu 1: medium - Features: [0 0 1 0 0 0]
Câu 2: easy - Features: [0 0 0 0 0 0]
Câu 3: easy - Features: [0 0 0 0 0 0]
Câu 4: medium - Features: [0 0 1 0 0 0]
Câu 5: medium - Features: [0 0 0 0 0 0]
Câu 6: easy - Features: [0 0 0 0 0 0]
Câu 7: medium - Features: [0 0 0 0 0 0]
Câu 8: easy - Features: [0 0 0 0 0 0]
Câu 9: medium - Features: [0 0 0 0 0 0]
Câu 10: medium - Features: [0 0 0 0 0 0]


In [10]:
# Cell 10 - Train difficulty model với evaluation
texts = []
for _, row in raw_data.iterrows():
    options_text = ' '.join(row['options']) if row['options'] else ''
    full_text = row['question'] + ' ' + options_text
    processed_text = clean_vietnamese_text(full_text, remove_stopwords=True, normalize=True)
    texts.append(processed_text)

X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, difficulties, test_size=0.2, random_state=42, stratify=difficulties
)

X_train_vec = difficulty_classifier.text_vectorizer.fit_transform(X_train_text)
X_test_vec = difficulty_classifier.text_vectorizer.transform(X_test_text)

difficulty_classifier.model.fit(X_train_vec, y_train)
y_pred = difficulty_classifier.model.predict(X_test_vec)

print("🎯 KẾT QUẢ DIFFICULTY CLASSIFICATION:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")

🎯 KẾT QUẢ DIFFICULTY CLASSIFICATION:
Accuracy: 0.6583
F1-Score: 0.6199


In [11]:
# Cell 11 - Confusion Matrix cho Difficulty
cm = confusion_matrix(y_test, y_pred)
labels = sorted(list(set(y_test) | set(y_pred)))

print("📊 CONFUSION MATRIX - DIFFICULTY:")
print(f"{'':>8}", end="")
for label in labels:
    print(f"{label:>8}", end="")
print()

for i, true_label in enumerate(labels):
    print(f"{true_label:>8}", end="")
    for j in range(len(labels)):
        print(f"{cm[i][j]:>8}", end="")
    print()

📊 CONFUSION MATRIX - DIFFICULTY:
            easy    hard  medium
    easy       3       3      17
    hard       0      50       7
  medium       0      14      26


In [12]:
# Cell 12 - Classification Report cho Difficulty
print("📋 CLASSIFICATION REPORT - DIFFICULTY:")
print(classification_report(y_test, y_pred))

📋 CLASSIFICATION REPORT - DIFFICULTY:
              precision    recall  f1-score   support

        easy       1.00      0.13      0.23        23
        hard       0.75      0.88      0.81        57
      medium       0.52      0.65      0.58        40

    accuracy                           0.66       120
   macro avg       0.76      0.55      0.54       120
weighted avg       0.72      0.66      0.62       120



In [13]:
# Cell 13 - TopicClassifier Class
class TopicClassifier:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 3), min_df=2, max_df=0.8)
        self.model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
        self.label_to_id = {}
        self.id_to_label = {}
        self.is_trained = False
        
        self.subject_topics = {
            'physics': ['Dao động cơ', 'Sóng cơ', 'Điện xoay chiều', 'Từ trường', 'Điện trường', 'Quang học', 'Cơ học', 'Nhiệt học'],
            'chemistry': ['Hóa hữu cơ', 'Este – Lipit', 'Điện phân', 'Cân bằng hóa học', 'Axit - Bazơ', 'Oxi hóa - Khử', 'Polime', 'Kim loại'],
            'biology': ['Di truyền học', 'Tiến hóa', 'Sinh thái học', 'Tế bào học', 'Sinh lý học', 'Phân loại sinh vật', 'Sinh học phân tử', 'Miễn dịch học']
        }
    
    def _create_topic_labels(self, data):
        topic_keywords = {
            'Dao động cơ': ['dao động', 'chu kỳ', 'tần số', 'biên độ', 'con lắc', 'lò xo'],
            'Điện xoay chiều': ['xoay chiều', 'điện áp hiệu dụng', 'dòng điện xoay chiều', 'máy biến áp'],
            'Hóa hữu cơ': ['ankan', 'anken', 'ankin', 'benzen', 'ancol', 'phenol', 'carbon'],
            'Di truyền học': ['gen', 'alen', 'NST', 'nhiễm sắc thể', 'ADN', 'ARN', 'đột biến', 'lai']
        }
        
        topics = []
        for _, row in data.iterrows():
            subject = row['subject']
            question_text = row['question'].lower()
            
            subject_topic_list = self.subject_topics.get(subject, [])
            best_topic = 'Khác'
            max_score = 0
            
            for topic in subject_topic_list:
                if topic in topic_keywords:
                    keywords = topic_keywords[topic]
                    score = sum(1 for keyword in keywords if keyword in question_text)
                    
                    if score > max_score:
                        max_score = score
                        best_topic = topic
            
            if best_topic == 'Khác':
                if subject == 'physics':
                    best_topic = 'Cơ học'
                elif subject == 'chemistry':
                    best_topic = 'Hóa hữu cơ'
                elif subject == 'biology':
                    best_topic = 'Tế bào học'
            
            topics.append(best_topic)
        
        return topics
    
    def get_topics_by_subject(self, subject):
        return self.subject_topics.get(subject, ['Khác'])

In [14]:
# Cell 14 - Khởi tạo và train TopicClassifier
topic_classifier = TopicClassifier()
print("🏷️ Đang tạo nhãn chủ đề...")
topics = topic_classifier._create_topic_labels(raw_data)

print("📊 PHÂN PHỐI CHỦ ĐỀ:")
topic_counts = pd.Series(topics).value_counts()
print(topic_counts.head(10))

🏷️ Đang tạo nhãn chủ đề...
📊 PHÂN PHỐI CHỦ ĐỀ:
Hóa hữu cơ         200
Cơ học             103
Tế bào học         100
Di truyền học      100
Dao động cơ         72
Điện xoay chiều     25
Name: count, dtype: int64


In [15]:
# Cell 15 - Phân tích topics theo subject
print("📋 CHỦ ĐỀ THEO MÔN HỌC:")
data_with_topics = raw_data.copy()
data_with_topics['topic'] = topics

for subject in ['physics', 'chemistry', 'biology']:
    subject_data = data_with_topics[data_with_topics['subject'] == subject]
    print(f"\n{subject.upper()}:")
    print(subject_data['topic'].value_counts().head(5))

📋 CHỦ ĐỀ THEO MÔN HỌC:

PHYSICS:
topic
Cơ học             103
Dao động cơ         72
Điện xoay chiều     25
Name: count, dtype: int64

CHEMISTRY:
topic
Hóa hữu cơ    200
Name: count, dtype: int64

BIOLOGY:
topic
Tế bào học       100
Di truyền học    100
Name: count, dtype: int64


In [16]:
# Cell 16 - Train topic classification model
topic_texts = []
for _, row in raw_data.iterrows():
    options_text = ' '.join(row['options']) if row['options'] else ''
    full_text = row['question'] + ' ' + options_text
    topic_texts.append(full_text)

unique_topics = sorted(list(set(topics)))
topic_classifier.label_to_id = {label: idx for idx, label in enumerate(unique_topics)}
topic_classifier.id_to_label = {idx: label for label, idx in topic_classifier.label_to_id.items()}

X_topic = topic_classifier.vectorizer.fit_transform(topic_texts)
y_topic = [topic_classifier.label_to_id[topic] for topic in topics]

X_train_topic, X_test_topic, y_train_topic, y_test_topic = train_test_split(
    X_topic, y_topic, test_size=0.2, random_state=42, stratify=y_topic
)

topic_classifier.model.fit(X_train_topic, y_train_topic)
y_pred_topic = topic_classifier.model.predict(X_test_topic)

print("🎯 KẾT QUẢ TOPIC CLASSIFICATION:")
print(f"Accuracy: {accuracy_score(y_test_topic, y_pred_topic):.4f}")
print(f"F1-Score: {f1_score(y_test_topic, y_pred_topic, average='weighted'):.4f}")

🎯 KẾT QUẢ TOPIC CLASSIFICATION:
Accuracy: 0.8667
F1-Score: 0.8648


In [17]:
# Cell 17 - SimilarQuestionFinder Class
class SimilarQuestionFinder:
    def __init__(self, data):
        self.data = data
        self.train_data, self.test_data = train_test_split(
            data, test_size=0.2, random_state=42, stratify=data['subject']
        )
        
        self.vectorizer = TfidfVectorizer(
            max_features=2000, ngram_range=(1, 2), min_df=2, max_df=0.85, sublinear_tf=True
        )
        
        self.question_vectors = None
        self._prepare_vectors()
    
    def _prepare_vectors(self):
        train_texts = []
        for _, row in self.train_data.iterrows():
            full_text = row['question'] + ' ' + ' '.join(row['options']) if row['options'] else row['question']
            processed = clean_vietnamese_text(full_text, remove_stopwords=True, normalize=True)
            train_texts.append(processed)
        
        self.vectorizer.fit(train_texts)
        
        all_texts = []
        for _, row in self.data.iterrows():
            full_text = row['question'] + ' ' + ' '.join(row['options']) if row['options'] else row['question']
            processed = clean_vietnamese_text(full_text, remove_stopwords=True, normalize=True)
            all_texts.append(processed)
            
        self.question_vectors = self.vectorizer.transform(all_texts)
    
    def find_similar_questions(self, current_question_id, n_similar=3):
        try:
            current_idx = None
            for idx, (_, row) in enumerate(self.data.iterrows()):
                if row['id'] == current_question_id:
                    current_idx = idx
                    break
            
            if current_idx is None:
                return []
            
            current_vector = self.question_vectors[current_idx]
            similarities = cosine_similarity(current_vector, self.question_vectors).flatten()
            
            similar_questions = []
            for idx, similarity in enumerate(similarities):
                if idx != current_idx:
                    question_data = self.data.iloc[idx]
                    similar_questions.append({
                        'question_data': question_data,
                        'similarity': similarity,
                        'index': idx
                    })
            
            similar_questions.sort(key=lambda x: x['similarity'], reverse=True)
            return similar_questions[:n_similar]
            
        except Exception:
            return []

In [18]:
# Cell 18 - Khởi tạo enhanced dataset
data_enhanced = raw_data.copy()
data_enhanced['difficulty'] = difficulties
data_enhanced['topic'] = topics

print("📊 ENHANCED DATASET:")
print(f"Shape: {data_enhanced.shape}")
print(f"Columns: {data_enhanced.columns.tolist()}")
data_enhanced.head()

📊 ENHANCED DATASET:
Shape: (600, 8)
Columns: ['id', 'question', 'options', 'answer', 'subject', 'explanation', 'difficulty', 'topic']


,id,question,options,answer,subject,explanation,difficulty,topic
0,MET_Bio_IE_2019_1,Có thể sử dụng hóa chất nào sau đây để phát hi...,"[A. Dung dịch NaCl., B. Dung dịch Ca(OH)2., C....",B,biology,Có thể được sử dụng Ca(OH)2 để phát hiện quá t...,medium,Tế bào học
1,MET_Bio_IE_2019_2,Động vật nào sau đây trao đổi khí với môi trườ...,"[A. Châu chấu., B. Sư tử., C. Chuột., D. Ếch đ...",A,biology,Châu chấu trao đổi khí với môi trường thông qu...,easy,Tế bào học
2,MET_Bio_IE_2019_3,Axit amin là đơn phân cấu tạo nên phân tử nào ...,"[A. ADN., B. mARN., C. tARN., D. Prôtêin.]",D,biology,Axit amin là đơn phân cấu tạo nên phân tử Prôt...,easy,Tế bào học
3,MET_Bio_IE_2019_4,Phân tử nào sau đây trực tiếp làm khuôn cho qu...,"[A. ADN., B. mARN., C. tARN., D. rARN.]",B,biology,Phân tử mARM trực tiếp làm khuôn cho quá trình...,medium,Tế bào học
4,MET_Bio_IE_2019_5,Một phân tử ADN ở vi khuẩn có 10% số nuclêôtit...,"[A. 10%., B. 30%., C. 20%., D. 40%.]",D,biology,"Theo nguyên tắc bổ sung A = T, G = X nên %A +%...",medium,Tế bào học


In [19]:
# Cell 19 - Khởi tạo SimilarQuestionFinder
similar_finder = SimilarQuestionFinder(data_enhanced)
print("🔍 Similar Question Finder đã được khởi tạo")
print(f"📊 Training data: {len(similar_finder.train_data)} questions")
print(f"📊 Test data: {len(similar_finder.test_data)} questions")
print(f"🔤 Vocabulary size: {len(similar_finder.vectorizer.vocabulary_)}")

🔍 Similar Question Finder đã được khởi tạo
📊 Training data: 480 questions
📊 Test data: 120 questions
🔤 Vocabulary size: 2000


In [20]:
# Cell 20 - Test similarity với sample questions
print("🧪 TEST SIMILARITY VỚI SAMPLE QUESTIONS:")
sample_ids = data_enhanced['id'].head(3).tolist()

for i, question_id in enumerate(sample_ids):
    current_q = data_enhanced[data_enhanced['id'] == question_id].iloc[0]
    print(f"\n--- Câu {i+1} ---")
    print(f"ID: {question_id}")
    print(f"Subject: {current_q['subject']}")
    print(f"Difficulty: {current_q['difficulty']}")
    print(f"Topic: {current_q['topic']}")
    print(f"Question: {current_q['question'][:80]}...")
    
    similar_questions = similar_finder.find_similar_questions(question_id, n_similar=2)
    if similar_questions:
        print("Similar questions:")
        for j, similar in enumerate(similar_questions):
            sim_q = similar['question_data']
            print(f"  {j+1}. Similarity: {similar['similarity']:.3f} | Subject: {sim_q['subject']} | {sim_q['question'][:50]}...")

🧪 TEST SIMILARITY VỚI SAMPLE QUESTIONS:

--- Câu 1 ---
ID: MET_Bio_IE_2019_1
Subject: biology
Difficulty: medium
Topic: Tế bào học
Question: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thả...
Similar questions:
  1. Similarity: 0.381 | Subject: biology | Câu 85. Nhóm thực vật nào sau đây xảy ra quá trình...
  2. Similarity: 0.300 | Subject: biology | Câu 109. Khi nói về hô hấp ở thực vật, có bao nhiê...

--- Câu 2 ---
ID: MET_Bio_IE_2019_2
Subject: biology
Difficulty: easy
Topic: Tế bào học
Question: Động vật nào sau đây trao đổi khí với môi trường thông qua hệ thống ống khí?...
Similar questions:
  1. Similarity: 0.616 | Subject: biology | Động vật nào sau đây hô hấp bằng hệ thống ống khí?...
  2. Similarity: 0.418 | Subject: biology | Câu 95. Sinh vật nào sau đây có quá trình trao đổi...

--- Câu 3 ---
ID: MET_Bio_IE_2019_3
Subject: biology
Difficulty: easy
Topic: Tế bào học
Question: Axit amin là đơn phân cấu tạo nên phân tử nào sau đây?...
Similar q

In [21]:
# Cell 21 - Evaluate similarity performance
test_sample = similar_finder.test_data.sample(min(50, len(similar_finder.test_data)), random_state=42)
same_subject_correct = 0
total_tests = 0
within_subject_similarity = []
cross_subject_similarity = []

print("📊 ĐÁNH GIÁ SIMILARITY PERFORMANCE:")

for _, test_question in test_sample.iterrows():
    similar_questions = similar_finder.find_similar_questions(test_question['id'], n_similar=3)
    
    if similar_questions:
        most_similar = similar_questions[0]
        if most_similar['question_data']['subject'] == test_question['subject']:
            same_subject_correct += 1
        
        for similar in similar_questions:
            sim_score = similar['similarity']
            if similar['question_data']['subject'] == test_question['subject']:
                within_subject_similarity.append(sim_score)
            else:
                cross_subject_similarity.append(sim_score)
        
        total_tests += 1

subject_accuracy = same_subject_correct / total_tests if total_tests > 0 else 0
print(f"Subject Accuracy: {subject_accuracy:.4f} ({same_subject_correct}/{total_tests})")

if within_subject_similarity:
    print(f"Within Subject Similarity: {np.mean(within_subject_similarity):.4f} ± {np.std(within_subject_similarity):.4f}")
if cross_subject_similarity:
    print(f"Cross Subject Similarity: {np.mean(cross_subject_similarity):.4f} ± {np.std(cross_subject_similarity):.4f}")

📊 ĐÁNH GIÁ SIMILARITY PERFORMANCE:
Subject Accuracy: 0.9800 (49/50)
Within Subject Similarity: 0.4839 ± 0.1394
Cross Subject Similarity: 0.2409 ± 0.0483


In [22]:
# Cell 22 - Utility functions cho quiz
def get_random_question(data, subject=None, year=None, difficulty=None, topic=None):
    filtered_data = data.copy()
    
    if subject:
        filtered_data = filtered_data[filtered_data['subject'] == subject]
    if year:
        filtered_data = filtered_data[filtered_data['id'].str.contains(str(year), na=False)]
    if difficulty and 'difficulty' in filtered_data.columns:
        filtered_data = filtered_data[filtered_data['difficulty'] == difficulty]
    if topic and 'topic' in filtered_data.columns:
        filtered_data = filtered_data[filtered_data['topic'] == topic]
    
    if len(filtered_data) == 0:
        return None
    
    return filtered_data.sample(1).iloc[0]

def check_answer(user_answer, correct_answer):
    return user_answer.upper().strip() == correct_answer.upper().strip()

print("✅ Utility functions đã được định nghĩa")

✅ Utility functions đã được định nghĩa


In [23]:
# Cell 23 - Test filter functionality
print("🔍 TEST FILTER FUNCTIONALITY:")

filter_tests = [
    {'subject': 'physics', 'difficulty': 'easy'},
    {'subject': 'chemistry', 'topic': 'Hóa hữu cơ'},
    {'difficulty': 'hard'},
    {'subject': 'biology', 'difficulty': 'medium'}
]

for i, filters in enumerate(filter_tests):
    print(f"\nTest {i+1}: {filters}")
    question = get_random_question(data_enhanced, **filters)
    if question is not None:
        print(f"  ✅ Found: {question['subject']} | {question['difficulty']} | {question['topic']}")
        print(f"  Question: {question['question'][:60]}...")
    else:
        print("  ❌ No questions found")

🔍 TEST FILTER FUNCTIONALITY:

Test 1: {'subject': 'physics', 'difficulty': 'easy'}
  ✅ Found: physics | easy | Cơ học
  Question: Câu 17. Khi nói về thuyết lượng tử ánh sáng, phát biểu nào s...

Test 2: {'subject': 'chemistry', 'topic': 'Hóa hữu cơ'}
  ✅ Found: chemistry | hard | Hóa hữu cơ
  Question: Thành phần chính của đá vôi là canxi cacbonat. Công thức của...

Test 3: {'difficulty': 'hard'}
  ✅ Found: chemistry | hard | Hóa hữu cơ
  Question: Để ${m}$ gam hỗn hợp ${E}$ gồm ${Al}$, ${Fe}$ và ${Cu}$ tron...

Test 4: {'subject': 'biology', 'difficulty': 'medium'}
  ✅ Found: biology | medium | Tế bào học
  Question: Một loài thực vật, xét 2 cặp NST kí hiệu là A, a và B, b. Cơ...


In [24]:
# Cell 24 - Demo quiz flow
print("🎮 DEMO QUIZ FLOW:")

# Get random question
current_question = get_random_question(data_enhanced, subject='physics')
if current_question is not None:
    print(f"📝 Câu hỏi ID: {current_question['id']}")
    print(f"📚 Môn: {current_question['subject']}")
    print(f"⭐ Độ khó: {current_question['difficulty']}")
    print(f"📖 Chủ đề: {current_question['topic']}")
    print(f"\n❓ Câu hỏi: {current_question['question']}")
    print(f"\n📋 Các đáp án:")
    for option in current_question['options']:
        print(f"  {option}")
    print(f"\n✅ Đáp án đúng: {current_question['answer']}")

🎮 DEMO QUIZ FLOW:
📝 Câu hỏi ID: MET_Phy_IE_2019_17
📚 Môn: physics
⭐ Độ khó: hard
📖 Chủ đề: Cơ học

❓ Câu hỏi: Câu 17. Đặt điện áp u = 200*cos (100*\pi*t) (V) vào hai đầu đoạn mạch gồm điện trở 100 Ohm, cuộn cảm thuần và tụ điện mắc nối tiếp. Biết trong đoạn mạch có cộng hưởng điện. Cường độ hiệu dụng của dòng điện trong đoạn mạch là

📋 Các đáp án:
  A. 2\sqrt{2} A.
  B. \sqrt{2} A.
  C. 2 A.
  D. 1 A.

✅ Đáp án đúng: B


In [25]:
# Cell 25 - Test answer checking
user_answers = ['A', 'B', 'C', 'D']
correct_answer = current_question['answer']

print(f"\n🧪 TEST ANSWER CHECKING:")
print(f"Correct answer: {correct_answer}")

for answer in user_answers:
    is_correct = check_answer(answer, correct_answer)
    result = "✅ Đúng" if is_correct else "❌ Sai"
    print(f"User answer '{answer}': {result}")


🧪 TEST ANSWER CHECKING:
Correct answer: B
User answer 'A': ❌ Sai
User answer 'B': ✅ Đúng
User answer 'C': ❌ Sai
User answer 'D': ❌ Sai


In [26]:
# Cell 26 - Test similar questions cho demo question
print(f"\n🔍 TÌM CÂU HỎI TƯƠNG TỰ:")
similar_questions = similar_finder.find_similar_questions(current_question['id'], n_similar=3)

if similar_questions:
    print(f"Tìm thấy {len(similar_questions)} câu hỏi tương tự:")
    for i, similar in enumerate(similar_questions, 1):
        similar_q = similar['question_data']
        similarity_score = similar['similarity']
        
        print(f"\n{i}. Độ tương đồng: {similarity_score:.3f}")
        print(f"   Subject: {similar_q['subject']} | Difficulty: {similar_q['difficulty']} | Topic: {similar_q['topic']}")
        print(f"   Question: {similar_q['question'][:80]}...")
        print(f"   Answer: {similar_q['answer']}")
else:
    print("❌ Không tìm thấy câu hỏi tương tự")


🔍 TÌM CÂU HỎI TƯƠNG TỰ:
Tìm thấy 3 câu hỏi tương tự:

1. Độ tương đồng: 0.630
   Subject: physics | Difficulty: hard | Topic: Điện xoay chiều
   Question: Câu 36. Đặt điện áp xoay chiều u = 60*\sqrt {2}*cos(100*\pi*t) (V) (t tính bằng ...
   Answer: A

2. Độ tương đồng: 0.615
   Subject: physics | Difficulty: hard | Topic: Cơ học
   Question: Câu 39. Cho đoạn mạch AB gồm cuộn cảm thuần L, điện trở R = 50 Ohm và tụ điện mắ...
   Answer: D

3. Độ tương đồng: 0.603
   Subject: physics | Difficulty: hard | Topic: Điện xoay chiều
   Question: Đặt điện áp xoay chiều u = U\sqrt {2}cos (\omega t) (\omega>0) vào hai đầu một đ...
   Answer: C


In [27]:
# Cell 27 - Score tracking demo
score_tracker = ScoreTracker()
print("📊 DEMO SCORE TRACKING:")

# Simulate some answers
test_results = [True, False, True, True, False, True, False, True, True, False]

for i, result in enumerate(test_results, 1):
    score_tracker.add_result(result)
    print(f"Câu {i}: {'✅' if result else '❌'} | Score: {score_tracker.correct}/{score_tracker.total} ({score_tracker.get_accuracy():.1f}%)")

📊 DEMO SCORE TRACKING:
Câu 1: ✅ | Score: 1/1 (100.0%)
Câu 2: ❌ | Score: 1/2 (50.0%)
Câu 3: ✅ | Score: 2/3 (66.7%)
Câu 4: ✅ | Score: 3/4 (75.0%)
Câu 5: ❌ | Score: 3/5 (60.0%)
Câu 6: ✅ | Score: 4/6 (66.7%)
Câu 7: ❌ | Score: 4/7 (57.1%)
Câu 8: ✅ | Score: 5/8 (62.5%)
Câu 9: ✅ | Score: 6/9 (66.7%)
Câu 10: ❌ | Score: 6/10 (60.0%)


In [28]:
# Cell 28 - Statistics summary
print("📈 THỐNG KÊ TỔNG QUAN HỆ THỐNG:")
print(f"📊 Tổng câu hỏi: {len(data_enhanced)}")
print(f"📚 Số môn học: {data_enhanced['subject'].nunique()}")
print(f"⭐ Số mức độ khó: {data_enhanced['difficulty'].nunique()}")
print(f"📖 Số chủ đề: {data_enhanced['topic'].nunique()}")

print(f"\n📊 Phân phối môn học:")
print(data_enhanced['subject'].value_counts())

print(f"\n⭐ Phân phối độ khó:")
print(data_enhanced['difficulty'].value_counts())

print(f"\n📖 Top 10 chủ đề phổ biến:")
print(data_enhanced['topic'].value_counts().head(10))

📈 THỐNG KÊ TỔNG QUAN HỆ THỐNG:
📊 Tổng câu hỏi: 600
📚 Số môn học: 3
⭐ Số mức độ khó: 3
📖 Số chủ đề: 6

📊 Phân phối môn học:
subject
biology      200
chemistry    200
physics      200
Name: count, dtype: int64

⭐ Phân phối độ khó:
difficulty
hard      282
medium    202
easy      116
Name: count, dtype: int64

📖 Top 10 chủ đề phổ biến:
topic
Hóa hữu cơ         200
Cơ học             103
Tế bào học         100
Di truyền học      100
Dao động cơ         72
Điện xoay chiều     25
Name: count, dtype: int64


In [29]:
# Cell 29 - Model performance summary
print("🤖 TỔNG KẾT PERFORMANCE CÁC MODEL:")

print(f"\n🎯 DIFFICULTY CLASSIFIER:")
print(f"  - Algorithm: Random Forest + TF-IDF")
print(f"  - Features: 15 NLP features + text vectorization")
print(f"  - Classes: {difficulty_counts.index.tolist()}")
print(f"  - Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"  - F1-Score (weighted): {f1_score(y_test, y_pred, average='weighted'):.4f}")

print(f"\n🏷️ TOPIC CLASSIFIER:")
print(f"  - Algorithm: Random Forest + TF-IDF")
print(f"  - Topics per subject: Physics({len(topic_classifier.subject_topics['physics'])}), Chemistry({len(topic_classifier.subject_topics['chemistry'])}), Biology({len(topic_classifier.subject_topics['biology'])})")
print(f"  - Total unique topics: {len(unique_topics)}")
print(f"  - Test Accuracy: {accuracy_score(y_test_topic, y_pred_topic):.4f}")
print(f"  - F1-Score (weighted): {f1_score(y_test_topic, y_pred_topic, average='weighted'):.4f}")

print(f"\n🔍 SIMILAR QUESTION FINDER:")
print(f"  - Algorithm: TF-IDF + Cosine Similarity")
print(f"  - Vocabulary size: {len(similar_finder.vectorizer.vocabulary_)}")
print(f"  - Subject accuracy: {subject_accuracy:.4f}")
if within_subject_similarity:
    print(f"  - Within-subject similarity: {np.mean(within_subject_similarity):.4f}")
if cross_subject_similarity:
    print(f"  - Cross-subject similarity: {np.mean(cross_subject_similarity):.4f}")

🤖 TỔNG KẾT PERFORMANCE CÁC MODEL:

🎯 DIFFICULTY CLASSIFIER:
  - Algorithm: Random Forest + TF-IDF
  - Features: 15 NLP features + text vectorization
  - Classes: ['hard', 'medium', 'easy']
  - Test Accuracy: 0.6583
  - F1-Score (weighted): 0.6199

🏷️ TOPIC CLASSIFIER:
  - Algorithm: Random Forest + TF-IDF
  - Topics per subject: Physics(8), Chemistry(8), Biology(8)
  - Total unique topics: 6
  - Test Accuracy: 0.8667
  - F1-Score (weighted): 0.8648

🔍 SIMILAR QUESTION FINDER:
  - Algorithm: TF-IDF + Cosine Similarity
  - Vocabulary size: 2000
  - Subject accuracy: 0.9800
  - Within-subject similarity: 0.4839
  - Cross-subject similarity: 0.2409


In [30]:
# Cell 30 - System readiness check
print("✅ KIỂM TRA SẴN SÀNG HỆ THỐNG:")

checks = [
    ("Data loaded", len(data_enhanced) > 0),
    ("Difficulty labels created", len(difficulties) == len(data_enhanced)),
    ("Topic labels created", len(topics) == len(data_enhanced)),
    ("Difficulty model trained", difficulty_classifier.model is not None),
    ("Topic model trained", topic_classifier.model is not None),
    ("Similar finder ready", similar_finder.question_vectors is not None),
    ("Enhanced dataset ready", 'difficulty' in data_enhanced.columns and 'topic' in data_enhanced.columns),
    ("Filter functions work", get_random_question(data_enhanced) is not None),
    ("Answer checking works", check_answer('A', 'A') == True),
    ("Score tracking works", score_tracker.get_accuracy() > 0)
]

all_passed = True
for check_name, passed in checks:
    status = "✅" if passed else "❌"
    print(f"{status} {check_name}")
    if not passed:
        all_passed = False

print(f"\n{'🎉 HỆ THỐNG SẴN SÀNG!' if all_passed else '⚠️ CÓ LỖI CẦN KHẮC PHỤC!'}")

if all_passed:
    print("\n🚀 Có thể chạy Streamlit app với lệnh:")
    print("streamlit run main.py")

✅ KIỂM TRA SẴN SÀNG HỆ THỐNG:
✅ Data loaded
✅ Difficulty labels created
✅ Topic labels created
✅ Difficulty model trained
✅ Topic model trained
✅ Similar finder ready
✅ Enhanced dataset ready
✅ Filter functions work
✅ Answer checking works
✅ Score tracking works

🎉 HỆ THỐNG SẴN SÀNG!

🚀 Có thể chạy Streamlit app với lệnh:
streamlit run main.py
